In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in the Open Food Facts dataset
df = pd.read_csv('en.openfoodfacts.org.products.tsv', sep='\t')

C:\Users\David\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# What does the data look like? Display it
pd.set_option("display.max_columns", len(df.columns))
df.head(2)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,NaN,NaN,28 g (1 ONZ),NaN,0.0,[ banan

## About the data:  
This data is hosted by Open Food Facts (https://world.openfoodfacts.org/) as an open source project. Anyone can contribute to the database and access it. The entire dataset contains 162 features ranging from product information to macro and micro nutrition information per 100 grams.

## Goal:
With this data, I want to create a nutritional heatmap. This data, however, does not show us what people actually eat on any sort of basis, but it does show us what may be readily available to each user who uploads information. Since some countries may dominate the dataset in terms of number of uploads, we have to be cognizant of the fact that aggregated information from countries with fewer entries may not be as accurate as countries with many entries. For example, a country with 100,000 entries and a mean sugar value per 100g of 20g is probably more reliable than a country with 50 entries and a mean sugar value per 100g of 35g, and the latter's true mean sugar content may not actually be much higher than that of the former. 

## Plan:
To accomplish the task set out above, I will aggregate the data by country and take mean values for sugar, protein, and fat per 100g as well as the number of entries used to calculate that mean. The data will then be visualized using Tableau.

=========================================================================================================================

First, we need to set up a function to take nutrition data and return mean values as well as number of values. Not all countries have nutrition data (for example, a country may have very few entries and there may not be any entries for the nutrient in question) so we set those mean values to 999999 so that they can be filtered out later.

In [4]:
def get_nutrient_info(df, country, nutrient):
    country_df = df[(df['countries_en'] == country)][(nutrient + '_100g')]
    num_null = len(country_df[country_df.isnull()])
    nutrient_list = list(country_df[country_df.notnull()])     
    num_not_null = len(nutrient_list)
    
    perc_not_null = num_not_null/(num_not_null+num_null)
    
    if len(nutrient_list) == 0:
        nutrient_mean = 999999
    else:
        nutrient_mean = np.mean(nutrient_list)
        
    return nutrient_mean, num_null, num_not_null, perc_not_null

There are three features that show us country information but we will focus on two, "countries" and "countries_en". We need to decide which feature to use. Unfortunately, there is not very much information regarding the features, so we have to do a bit of guesswork. Let's explore after setting up a dictionary per country where "countries_en" are the keys and related "countries" are the values:

In [5]:
# Create a dictionary where "countries_en" are the keys and "countries" are the values
all_country_list = []
for country in list(df['countries_en'].unique()):
    try:
        if len(country.split(',')) == 1:
            all_country_list.append(country)
    except:
        print('Error:', country)
        
all_country_dict = {}
for country in all_country_list:
    all_country_dict[country] = list(df[df['countries_en'] == country]['countries'].unique())   

Error: nan


After some wrangling, it becomes apparent that "countries" is far more variable than "countries_en" where "countries_en" serves as an aggregation of "countries". For example, as shown below, the United States may be represented as any of the 13 different names in the "countries" feature, but it is represented as a whole as "United States" in the "countries_en" feature. We can imagine how difficult it might be to parse through each entry in "countries" to determine what all the different levels refer to, so keeping it high level with "countries_en" is in our best interest.

We will continue by aggregating by all unique countries in "countries_en".

In [6]:
# What are the different names for United States in "countries"?
all_country_dict['United States']

['US',
 'United States',
 'Usa',
 'États-Unis',
 'en:US',
 'USA',
 'united-states',
 'United states',
 'Etats-Unis',
 'United States of America',
 'Estados Unidos',
 'en:US, US',
 'Vereinigte Staaten von Amerika']

Now we can move on to extracting average nutrient values per country as well as how many relevent entries have been made by users:

In [7]:
# Create an empty dictionary for each country
nutrient_dict = {}
for country in list(all_country_dict.keys()):
    nutrient_dict[country] = []

# Create a list of nutrients of interest
nutrients = ['sugars', 'proteins', 'fat']

# Fill in the empty dictionary with all nurition info and entry counts per country
for nutrient in nutrients:
    for country in list(all_country_dict.keys()):
        nutrient_dict[country] = nutrient_dict[country] + list(get_nutrient_info(df, country, nutrient))      

In [8]:
# Create feature names before transforming from dictionary to dataframe
df_columns = []
for nutrient in nutrients:
    df_columns = df_columns + [str('avg_' + nutrient),
                               str(nutrient + '_nulls'),
                               str(nutrient + '_not_nulls'),
                               str(nutrient + '_perc_not_nulls')]
    
# Transform the dictionary into a dataframe and display
nutrient_df = pd.DataFrame.from_dict(nutrient_dict).T
nutrient_df = nutrient_df.reset_index(drop=False)
nutrient_df.columns = ['country'] + df_columns

nutrient_df.head()

,country,avg_sugars,sugars_nulls,sugars_not_nulls,sugars_perc_not_nulls,avg_proteins,proteins_nulls,proteins_not_nulls,proteins_perc_not_nulls,avg_fat,fat_nulls,fat_not_nulls,fat_perc_not_nulls
0,Albania,999999.000000,8.0,0.0,0.000000,999999.000000,8.0,0.0,0.000000,999999.000000,8.0,0.0,0.000000
1,Algeria,0.700000,145.0,1.0,0.006849,3.866667,143.0,3.0,0.020548,6.556667,143.0,3.0,0.020548
2,Andorra,8.923333,7.0,6.0,0.461538,9.531667,7.0,6.0,0.461538,11.093333,7.0,6.0,0.461538
3,Angola,999999.000000,2.0,0.0,0.000000,999999.000000,2.0,0.0,0.000000,999999.000000,2.0,0.0,0.000000
4,Argentina,29.582500,50.0,4.0,0.074074,2.761000,44.0,10.0,0.185185,6.636364,43.0,11.0,0.203704


In [9]:
print('We have {} countries in the dataset.'.format(len(nutrient_df)))

We have 180 countries in the dataset.


We see those 999999 values that we inserted for countries without any valid entries, and so now we have to filter those out:

In [10]:
cleaned_nutrient_df = nutrient_df[np.logical_not((nutrient_df == 999999).any(1))]

print('We have {} countries in the cleaned dataset.'.format(len(cleaned_nutrient_df)))

We have 93 countries in the cleaned dataset.


We also see that there are a few countries that have valid entries, but are either too broad (European Union), mislabeled, or entirely not in English. There are also entries that we may be able to insert into a broader category (Other-japon into Japan, for example) but, for the sake of consistency and due to the low number of these observations, we will not be adding them. We drop those entries below:

In [11]:
drop_countries = ['European Union', 'Other-japon', 'Other-turquie',
                  'World','fr:Quebec', 'الإمارات-العربية-المتحدة' ,'السعودية' ,'تونس', 'سلطنة-عمان', 'لبنان', '香港']

cleaned_nutrient_df = cleaned_nutrient_df[np.logical_not(cleaned_nutrient_df['country']\
                                                         .isin(drop_countries))]
print('We have {} countries in the further cleaned dataset.'.format(len(cleaned_nutrient_df)))

We have 82 countries in the further cleaned dataset.


In [12]:
# Display the cleaned dataset
cleaned_nutrient_df.head()

,country,avg_sugars,sugars_nulls,sugars_not_nulls,sugars_perc_not_nulls,avg_proteins,proteins_nulls,proteins_not_nulls,proteins_perc_not_nulls,avg_fat,fat_nulls,fat_not_nulls,fat_perc_not_nulls
1,Algeria,0.700000,145.0,1.0,0.006849,3.866667,143.0,3.0,0.020548,6.556667,143.0,3.0,0.020548
2,Andorra,8.923333,7.0,6.0,0.461538,9.531667,7.0,6.0,0.461538,11.093333,7.0,6.0,0.461538
4,Argentina,29.582500,50.0,4.0,0.074074,2.761000,44.0,10.0,0.185185,6.636364,43.0,11.0,0.203704
6,Aruba,8.000000,6.0,1.0,0.142857,8.000000,6.0,1.0,0.142857,17.300000,6.0,1.0,0.142857
7,Australia,13.872540,1651.0,512.0,0.236708,6.751748,1671.0,492.0,0.227462,11.138834,1656.0,507.0,0.234397


We now see the phenomenon described above: Some countries with few entries may dominate countries with more entries that have more accurate information. We see, for example, Pakistan has one non-null entry for sugar but that single entry has 56.8g sugars per 100g, and will unfairly dominate over other countries:

In [13]:
cleaned_nutrient_df[cleaned_nutrient_df['country'] == 'Pakistan']

,country,avg_sugars,sugars_nulls,sugars_not_nulls,sugars_perc_not_nulls,avg_proteins,proteins_nulls,proteins_not_nulls,proteins_perc_not_nulls,avg_fat,fat_nulls,fat_not_nulls,fat_perc_not_nulls
115,Pakistan,56.8,7.0,1.0,0.125,10.5,7.0,1.0,0.125,19.0,7.0,1.0,0.125


To account for this, we filter over all nutrients for any countries that have over 150 valid entries:

In [14]:
final_nutrient_df = cleaned_nutrient_df[(cleaned_nutrient_df['sugars_not_nulls'] > 150) 
                                        & (cleaned_nutrient_df['proteins_not_nulls'] > 150)
                                        & (cleaned_nutrient_df['fat_not_nulls'] > 150)]

final_nutrient_df

,country,avg_sugars,sugars_nulls,sugars_not_nulls,sugars_perc_not_nulls,avg_proteins,proteins_nulls,proteins_not_nulls,proteins_perc_not_nulls,avg_fat,fat_nulls,fat_not_nulls,fat_perc_not_nulls
7,Australia,13.872540,1651.0,512.0,0.236708,6.751748,1671.0,492.0,0.227462,11.138834,1656.0,507.0,0.234397
15,Belgium,14.642511,2116.0,556.0,0.208084,6.140062,2076.0,596.0,0.223054,14.231998,2071.0,601.0,0.224925
25,Canada,16.546304,723.0,191.0,0.208972,7.291710,718.0,196.0,0.214442,13.132672,716.0,198.0,0.216630
48,France,13.231082,35080.0,70041.0,0.666289,7.787255,33479.0,71642.0,0.681519,13.324960,49606.0,55515.0,0.528106
53,Germany,13.808692,3465.0,4619.0,0.571376,8.672780,3436.0,4648.0,0.574963,15.550687,3398.0,4686.0,0.579664
73,Italy,12.912491,1006.0,261.0,0.205998,7.214198,843.0,424.0,0.334649,10.350024,844.0,423.0,0.333860
104,Netherlands,20.141245,347.0,241.0,0.409864,7.123213,339.0,249.0,0.423469,15.258960,338.0,250.0,0.425170
122,Portugal,12.849612,371.0,335.0,0.474504,6.888232,344.0,362.0,0.512748,13.342732,340.0,366.0,0.518414
144,Spain,11.957492,2715.0,2577.0,0.486961,5.901832,2265.0,3027.0,0.571995,12.843836,2254.0,3038.0,0.574074
147,Switzerland,15.535237,5018.0,9929.0,0.664280,6.984364,4226.0,10721.0,0.717268,13.406772,4174.0,10773.0,0.720747


In [15]:
print('We are left with {} countries for visualization.'.format(len(final_nutrient_df)))

We are left with 12 countries for visualization.


We now have the data we need for visualization purposes, and will write it to a CSV file:

In [16]:
# Write the data to a file
final_nutrient_df.to_csv('open_food_data_analysis.csv', sep=',')

## Result:

Overall, we find the following:
1. For sugar, food from the Netherlands tends to have the highest mean sugar per 100g at 20.141g. Spain's food tends to have the least at 11.957g.
2. For protein, food from Germany tends to have the highest mean protein per 100g at 8.673g. Spain's food tends to have the least at 5.902g.
3. For fat, food from Germany tends to have the highest mean fat per 100g at 15.551g. Italy's food tends to have the least at 10.350g.

We can make this information much more visually appealing using Tableau. The visualization can be found here: https://public.tableau.com/profile/dpalbrecht#!/vizhome/OpenFoodFacts_0/Sugar?publish=yes